###Unstructured Files

In [0]:
%fs ls dbfs:/FileStore/FileStore/shared_uploads/images/

path,name,size
dbfs:/FileStore/FileStore/shared_uploads/images/DSC_0100.jpg,DSC_0100.jpg,41168
dbfs:/FileStore/FileStore/shared_uploads/images/DSC_0101.jpg,DSC_0101.jpg,42683
dbfs:/FileStore/FileStore/shared_uploads/images/DSC_0104.jpg,DSC_0104.jpg,112037
dbfs:/FileStore/FileStore/shared_uploads/images/DSC_0105.jpg,DSC_0105.jpg,134072
dbfs:/FileStore/FileStore/shared_uploads/images/DSC_0106.jpg,DSC_0106.jpg,128454


In [0]:
raw_df = spark.read.format("image").load("/FileStore/FileStore/shared_uploads/images/")
raw_df.printSchema()
image_df = raw_df.select("image.origin", "image.height", "image.width", "image.nChannels", "image.mode", "image.data")
image_df.write.option("header", "true").mode("overwrite").csv("/tmp/data-lake/images.csv")

root
-- image: struct (nullable = true)
 |-- origin: string (nullable = true)
 |-- height: integer (nullable = true)
 |-- width: integer (nullable = true)
 |-- nChannels: integer (nullable = true)
 |-- mode: integer (nullable = true)
 |-- data: binary (nullable = true)

In [0]:
from pyspark.sql.functions import max, lit
temp_df = final_df.withColumn("max_width", lit(final_df.agg(max("width")).first()[0]))
temp_df.where("width == max_width").show()

+--------------------+------+-----+---------+----+--------------------+---------+
 origin|height|width|nChannels|mode| data|max_width|
+--------------------+------+-----+---------+----+--------------------+---------+
dbfs:/FileStore/F...| 371| 1530| 3| 16|[FF FF FF FF FF F...| 1530|
+--------------------+------+-----+---------+----+--------------------+---------+

In [0]:
%fs ls dbfs:/tmp/data-lake/images.csv

path,name,size
dbfs:/tmp/data-lake/images.csv/_SUCCESS,_SUCCESS,0
dbfs:/tmp/data-lake/images.csv/_committed_2045495548478262677,_committed_2045495548478262677,464
dbfs:/tmp/data-lake/images.csv/_committed_3489961170606606401,_committed_3489961170606606401,909
dbfs:/tmp/data-lake/images.csv/_started_2045495548478262677,_started_2045495548478262677,0
dbfs:/tmp/data-lake/images.csv/_started_3489961170606606401,_started_3489961170606606401,0
dbfs:/tmp/data-lake/images.csv/part-00000-tid-3489961170606606401-1b25cdd6-b871-481b-95b2-b2860e4e9e9a-0-1-c000.csv,part-00000-tid-3489961170606606401-1b25cdd6-b871-481b-95b2-b2860e4e9e9a-0-1-c000.csv,127
dbfs:/tmp/data-lake/images.csv/part-00001-tid-3489961170606606401-1b25cdd6-b871-481b-95b2-b2860e4e9e9a-1-1-c000.csv,part-00001-tid-3489961170606606401-1b25cdd6-b871-481b-95b2-b2860e4e9e9a-1-1-c000.csv,126
dbfs:/tmp/data-lake/images.csv/part-00002-tid-3489961170606606401-1b25cdd6-b871-481b-95b2-b2860e4e9e9a-2-1-c000.csv,part-00002-tid-3489961170606606401-1b25cdd6-b871-481b-95b2-b2860e4e9e9a-2-1-c000.csv,127
dbfs:/tmp/data-lake/images.csv/part-00003-tid-3489961170606606401-1b25cdd6-b871-481b-95b2-b2860e4e9e9a-3-1-c000.csv,part-00003-tid-3489961170606606401-1b25cdd6-b871-481b-95b2-b2860e4e9e9a-3-1-c000.csv,126
dbfs:/tmp/data-lake/images.csv/part-00004-tid-3489961170606606401-1b25cdd6-b871-481b-95b2-b2860e4e9e9a-4-1-c000.csv,part-00004-tid-3489961170606606401-1b25cdd6-b871-481b-95b2-b2860e4e9e9a-4-1-c000.csv,126


###Semi-Structured

In [0]:
csv_df = spark.read.options(header="true", inferSchema="true").csv("/tmp/data-lake/images.csv")
csv_df.printSchema()
csv_df.show()

root
-- origin: string (nullable = true)
-- height: integer (nullable = true)
-- width: integer (nullable = true)
-- nChannels: integer (nullable = true)
-- mode: integer (nullable = true)
-- data: string (nullable = true)

+--------------------+------+-----+---------+----+-----------+
 origin|height|width|nChannels|mode| data|
+--------------------+------+-----+---------+----+-----------+
dbfs:/FileStore/F...| 279| 1480| 3| 16|[B@359e1730|
dbfs:/FileStore/F...| 323| 1504| 3| 16|[B@43ab39bf|
dbfs:/FileStore/F...| 371| 1530| 3| 16| [B@67188ba|
dbfs:/FileStore/F...| 250| 766| 3| 16|[B@1fc4cb4a|
dbfs:/FileStore/F...| 244| 617| 3| 16| [B@e4531cf|
+--------------------+------+-----+---------+----+-----------+

In [0]:
from pyspark.sql.functions import max, lit
temp_df = csv_df.withColumn("max_width", lit(csv_df.agg(max("width")).first()[0]))
temp_df.where("width == max_width").show()

+--------------------+------+-----+---------+----+----------+---------+
 origin|height|width|nChannels|mode| data|max_width|
+--------------------+------+-----+---------+----+----------+---------+
dbfs:/FileStore/F...| 371| 1530| 3| 16|[B@67188ba| 1530|
+--------------------+------+-----+---------+----+----------+---------+

###Structured Data

In [0]:
final_df.write.parquet("/tmp/data-lake/images.parquet")
parquet_df = spark.read.parquet("/tmp/data-lake/images.parquet")
parquet_df.printSchema()
display(parquet_df)

root
-- origin: string (nullable = true)
-- height: integer (nullable = true)
-- width: integer (nullable = true)
-- nChannels: integer (nullable = true)
-- mode: integer (nullable = true)
-- data: binary (nullable = true)

origin,height,width,nChannels,mode,data
dbfs:/FileStore/FileStore/shared_uploads/images/DSC_0106.jpg,323,1504,3,16,Binary image data placeholder. Access the image data field directly to view raw binary data.
dbfs:/FileStore/FileStore/shared_uploads/images/DSC_0105.jpg,279,1480,3,16,Binary image data placeholder. Access the image data field directly to view raw binary data.
dbfs:/FileStore/FileStore/shared_uploads/images/DSC_0104.jpg,371,1530,3,16,Binary image data placeholder. Access the image data field directly to view raw binary data.
dbfs:/FileStore/FileStore/shared_uploads/images/DSC_0100.jpg,250,766,3,16,Binary image data placeholder. Access the image data field directly to view raw binary data.
dbfs:/FileStore/FileStore/shared_uploads/images/DSC_0101.jpg,244,617,3,16,Binary image data placeholder. Access the image data field directly to view raw binary data.


In [0]:
from pyspark.sql.functions import max, lit
temp_df = parquet_df.withColumn("max_width", lit(parquet_df.agg(max("width")).first()[0]))
temp_df.where("width == max_width").show()

+--------------------+------+-----+---------+----+--------------------+---------+
 origin|height|width|nChannels|mode| data|max_width|
+--------------------+------+-----+---------+----+--------------------+---------+
dbfs:/FileStore/F...| 371| 1530| 3| 16|[FF FF FF FF FF F...| 1530|
+--------------------+------+-----+---------+----+--------------------+---------+

In [0]:
%fs ls /tmp/data-lake/images.parquet

path,name,size
dbfs:/tmp/data-lake/images.parquet/_SUCCESS,_SUCCESS,0
dbfs:/tmp/data-lake/images.parquet/_committed_3764051096931124086,_committed_3764051096931124086,524
dbfs:/tmp/data-lake/images.parquet/_started_3764051096931124086,_started_3764051096931124086,0
dbfs:/tmp/data-lake/images.parquet/part-00000-tid-3764051096931124086-663ac7ee-c635-4e39-b4b3-e7919f10b355-176-1-c000.snappy.parquet,part-00000-tid-3764051096931124086-663ac7ee-c635-4e39-b4b3-e7919f10b355-176-1-c000.snappy.parquet,1101459
dbfs:/tmp/data-lake/images.parquet/part-00001-tid-3764051096931124086-663ac7ee-c635-4e39-b4b3-e7919f10b355-177-1-c000.snappy.parquet,part-00001-tid-3764051096931124086-663ac7ee-c635-4e39-b4b3-e7919f10b355-177-1-c000.snappy.parquet,1228108
dbfs:/tmp/data-lake/images.parquet/part-00002-tid-3764051096931124086-663ac7ee-c635-4e39-b4b3-e7919f10b355-178-1-c000.snappy.parquet,part-00002-tid-3764051096931124086-663ac7ee-c635-4e39-b4b3-e7919f10b355-178-1-c000.snappy.parquet,1042820
dbfs:/tmp/data-lake/images.parquet/part-00003-tid-3764051096931124086-663ac7ee-c635-4e39-b4b3-e7919f10b355-179-1-c000.snappy.parquet,part-00003-tid-3764051096931124086-663ac7ee-c635-4e39-b4b3-e7919f10b355-179-1-c000.snappy.parquet,290031
dbfs:/tmp/data-lake/images.parquet/part-00004-tid-3764051096931124086-663ac7ee-c635-4e39-b4b3-e7919f10b355-180-1-c000.snappy.parquet,part-00004-tid-3764051096931124086-663ac7ee-c635-4e39-b4b3-e7919f10b355-180-1-c000.snappy.parquet,307002


####Clean-up 
**Databricks Community Edition** has a limitation of 10000 files and 10 GB of storage in DBFS.<br>
So it is prudent to clean-up any intermediate datasets created on DBFS that we do not intent to use at a later time.

In [0]:
%fs rm -r /tmp/data-lake/images.parquet

res0: Boolean = false